In [467]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [468]:
import json
from inspect import signature

In [490]:
from ollama import Client
from IPython.display import display_markdown, display_latex

client = Client(host="http://localhost:11434")

model_name = "qwen2.5:7b"

# Pull a model
result = client.pull(model=model_name)
result

{'status': 'success'}

### Define Tools

In [470]:
def summation(x: int, y: int = 0) -> int:
    """
    Add two integers.

    Parameters
    ----------
    x : int
        The first integer to be added.
    y : int, optional
        The second integer to be added (default is 0).

    Returns
    -------
    int
        The sum of `x` and `y`.
    """
    return x + y
    

def multiplication(x: int, y: int) -> int:
    """
    Multiply two integers.

    Parameters
    ----------
    x : int
        The first integer to be multiplied.
    y : int
        The second integer to be multiplied.

    Returns
    -------
    int
        The product of `x` and `y`.
    """
    return x * y


In [471]:
import xml.etree.ElementTree as ET
import json

def parse_tool_call(tool_call_str: str) -> dict:
    """
    Parse a tool_call XML string and return the corresponding dictionary.

    Parameters
    ----------
    tool_call_str : str
        A string containing the XML with the tool call.

    Returns
    -------
    dict
        A dictionary containing the parsed JSON from the tool_call tag.
    """
    # Parse a string XML
    root = ET.fromstring(tool_call_str)

    # Extrair o conteúdo da tag <tool_call>
    json_content = root.text.strip()

    # Converter o conteúdo JSON para um dicionário Python
    tool_call_dict = json.loads(json_content)

    return tool_call_dict


In [472]:
get_json_schema(multiplication)

'{\n    "name": "multiplication",\n    "description": "\\n    Multiply two integers.\\n\\n    Parameters\\n    ----------\\n    x : int\\n        The first integer to be multiplied.\\n    y : int\\n        The second integer to be multiplied.\\n\\n    Returns\\n    -------\\n    int\\n        The product of `x` and `y`.\\n    ",\n    "parameters": {\n        "properties": {\n            "x": {\n                "type": "int"\n            },\n            "y": {\n                "type": "int"\n            }\n        }\n    }\n}'

In [473]:
import json
from inspect import signature

class ToolRegistry:
    """
    A class to manage and register available tools (functions).
    
    This registry allows the user to register tools, retrieve them, and generate
    JSON schemas for each tool's function signature.
    """

    def __init__(self):
        """
        Initialize a ToolRegistry instance.
        
        Attributes
        ----------
        tools : dict
            A dictionary that stores tools with their names as keys and functions as values.
        """
        self.tools = {}

    def register(self, name: str, func):
        """
        Register a new function as a tool.
        
        Parameters
        ----------
        name : str
            The name of the tool to be registered.
        func : function
            The function to register as a tool.
        
        Raises
        ------
        TypeError
            If `func` is not callable.
        """
        if not callable(func):
            raise TypeError(f"The tool '{name}' must be a callable function.")
        self.tools[name] = func

    def get_function(self, name: str):
        """
        Get a registered function by its name.
        
        Parameters
        ----------
        name : str
            The name of the tool (function) to retrieve.
        
        Returns
        -------
        function or None
            The function registered under the given name, or None if not found.
        """
        return self.tools.get(name)

    def get_json_schema(self, func):
        """
        Generate the JSON schema for a given function.
        
        Parameters
        ----------
        func : function
            The function for which to generate the JSON schema.
        
        Returns
        -------
        str
            A JSON-formatted string representing the function's signature schema.
        
        Raises
        ------
        ValueError
            If the provided object is not a callable function.
        """
        if not callable(func):
            raise ValueError("The argument must be a callable function.")
        
        sigs = signature(func)
        desc = func.__doc__ or "No description provided."
        name = func.__name__

        schema = {
            "name": name,
            "description": desc,
            "parameters": {
                "properties": {
                    param_name: {"type": param.annotation.__name__ if param.annotation != param.empty else "unknown"}
                    for param_name, param in sigs.parameters.items()
                }
            }
        }

        return json.dumps(schema, indent=4)

    def generate_tools_schema(self):
        """
        Generate JSON schemas for all registered tools.
        
        Returns
        -------
        str
            A concatenation of JSON schemas for all tools, each schema on a new line.
        """
        schemas = [self.get_json_schema(func) for func in self.tools.values()]
        return "\n".join(schemas)


In [474]:
# Instanciar o registro de ferramentas
tool_registry = ToolRegistry()

# Registrar as funções
tool_registry.register("summation", summation)
tool_registry.register("multiplication", multiplication)

tools_schemas_string = tool_registry.generate_tools_schema()

In [486]:
TOOL_PROMPT = """You are a function calling AI model. You may call one or more functions to assist with the user query. You are provided with function signatures within <tools></tools> XML tags:

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:

<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call>

Here are the available tools:

<tools>
%s
</tools>
""" % tools_schemas_string

In [487]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_PROMPT,
    }
]

agent_chat_history = []

USER_PROMPT = "Can you multiple 25 by 12?"

tool_chat_history.append(
    {"role": "user", "content": USER_PROMPT}
)
agent_chat_history.append(
    {"role": "user", "content": USER_PROMPT}
)

tool_call_request = client.chat(model=model_name, messages=tool_chat_history)

tool_call_str = tool_call_request['message']['content']
tool_info = parse_tool_call(tool_call_str)

tool_call_result = tool_registry.get_function(tool_info['name'])(**tool_info['arguments'])

agent_chat_history.append({
    "role": "user",
    "content": f"Observation: {tool_call_result}"
})

tool_response = client.chat(model=model_name, messages=agent_chat_history)

response_using_tool = tool_response['message']['content']

In [496]:
response_using_tool 

"Certainly! The product of 25 and 12 is indeed 300. Here's the calculation:\n\n\\[ 25 \\times 12 = 300 \\]\n\nIs there anything else you'd like to know about this multiplication or any other math-related questions?"